# 07 - Authentication

## Use Case: User Registration & Login Flow

SurrealDB has built-in JWT authentication. The ORM provides a high-level API for:

- **Signup** — Register a new user and receive a JWT token
- **Signin** — Authenticate an existing user and receive a JWT token
- **Token Validation** — Verify a JWT token and retrieve the user

Authentication uses SurrealDB's `DEFINE ACCESS` statement, which defines how users
can sign up and sign in. The ORM handles token management, ephemeral connections
(so auth calls never corrupt the root connection), and automatic user model hydration.

This notebook covers:

1. **User Model with Auth** — Configuring a model for authentication
2. **Signup** — Creating new users with `User.signup()`
3. **Signin** — Authenticating with `User.signin()`
4. **Token Validation** — `authenticate_token()` and `validate_token()`
5. **Configurable Access Name** — Custom access definitions

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Python packages installed (`uv sync` in the project root)

In [ ]:
# Setup: add project root to path and configure the connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## User Model with Authentication

To enable authentication on a model, mix in `AuthenticatedUserMixin` and set
`table_type=TableType.USER` in the config.

Key configuration:
- **`table_type=TableType.USER`** — Marks this table as a user table
- **`access_name`** — The name of the `DEFINE ACCESS` rule in SurrealDB (defaults to `"{table}_auth"`)
- **`Encrypted` field** — Passwords are automatically hashed with argon2 by SurrealDB

In [ ]:
# Define the User model with authentication support
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict
from src.surreal_orm.types import TableType
from src.surreal_orm.fields import Encrypted
from src.surreal_orm.auth import AuthenticatedUserMixin


class User(AuthenticatedUserMixin, BaseSurrealModel):
    model_config = SurrealConfigDict(
        table_name="users",
        table_type=TableType.USER,
        access_name="account",  # Custom DEFINE ACCESS name (default would be "users_auth")
    )

    id: str | None = None
    email: str
    password: Encrypted  # Auto-hashed with argon2 by SurrealDB
    name: str
    role: str = "user"


print("User model defined with AuthenticatedUserMixin")
print(f"Access name: {User.model_config.get('access_name', 'users_auth')}")

## Important: DEFINE ACCESS Required

Before `signup()` or `signin()` can work, you must define an access rule in SurrealDB.
This is typically done once during database setup or in a migration.

The access rule tells SurrealDB:
1. **How to create a user on signup** (which fields to store)
2. **How to authenticate on signin** (which fields to check)
3. **Token duration** (how long the JWT is valid)

Below we create the access definition using a raw query. In production, you would
include this in your migration files.

In [ ]:
# Create the DEFINE ACCESS rule that enables signup/signin
# This must match the access_name in the model config ("account")
access_query = """
DEFINE ACCESS account ON DATABASE TYPE RECORD
    SIGNUP (
        CREATE users SET
            email = $email,
            password = crypto::argon2::generate($password),
            name = $name,
            role = "user"
    )
    SIGNIN (
        SELECT * FROM users WHERE email = $email
            AND crypto::argon2::compare(password, $password)
    )
    DURATION FOR SESSION 24h, FOR TOKEN 1h;
"""

await User.raw_query(access_query)
print("DEFINE ACCESS 'account' created successfully.")
print("Users can now sign up and sign in via the ORM.")

## 1. Signup — User Registration

`User.signup()` creates a new user via SurrealDB's access layer and returns
a tuple of `(user_instance, jwt_token)`.

Under the hood:
1. An **ephemeral connection** is created (not the root connection)
2. The signup RPC is called with the provided credentials
3. SurrealDB executes the SIGNUP query from DEFINE ACCESS
4. The user record is created and a JWT token is returned
5. The ephemeral connection is closed (root connection is unaffected)

In [ ]:
# Register a new user
# signup() returns (user_instance, jwt_token)
alice, alice_token = await User.signup(
    email="alice@example.com",
    password="securepassword123",
    name="Alice Johnson",
)

print(f"User created: {alice.name} ({alice.email})")
print(f"User ID: {alice.id}")
print(f"JWT token (first 50 chars): {alice_token[:50]}...")
print(f"Role: {alice.role}")

In [ ]:
# Register a second user to demonstrate multiple accounts
bob, bob_token = await User.signup(
    email="bob@example.com",
    password="anothersecret456",
    name="Bob Smith",
)

print(f"User created: {bob.name} ({bob.email})")
print(f"Each user gets a unique token — tokens are different: {alice_token[:20]} vs {bob_token[:20]}")

## 2. Signin — User Login

`User.signin()` authenticates an existing user and returns `(user_instance, jwt_token)`.

Like signup, this uses an ephemeral connection so it never interferes with
other ORM operations running concurrently.

In [ ]:
# Sign in as Alice
alice_login, login_token = await User.signin(
    email="alice@example.com",
    password="securepassword123",
)

print(f"Signed in as: {alice_login.name} ({alice_login.email})")
print(f"New JWT token (different from signup token): {login_token[:50]}...")
print(f"Same user ID: {alice_login.id == alice.id}")

## 3. Token Validation

The ORM provides two methods for token validation:

### `authenticate_token(token)` — Full Validation
- Validates the JWT token via SurrealDB
- Fetches the complete user record from the database
- Returns `(user_instance, record_id)` or `None` if invalid
- Use when you need the full user object (e.g., for authorization checks)

### `validate_token(token)` — Lightweight Validation
- Validates the JWT token via SurrealDB
- Returns just the `record_id` string, or `None` if invalid
- Does NOT fetch the user record (faster)
- Use for simple authentication gates (e.g., middleware)

In [ ]:
# Full token validation: returns user instance + record_id
result = await User.authenticate_token(login_token)

if result:
    user, record_id = result
    print(f"Token valid! User: {user.name} ({user.email})")
    print(f"Record ID: {record_id}")
    print(f"Role: {user.role}")
else:
    print("Token is invalid or expired.")

In [ ]:
# Lightweight token validation: returns just the record_id (no DB fetch)
record_id = await User.validate_token(login_token)

if record_id:
    print(f"Token valid! Record ID: {record_id}")
    print("(User object was NOT fetched — use this for fast auth gates)")
else:
    print("Token is invalid or expired.")

In [ ]:
# Demonstrate invalid token handling
invalid_result = await User.validate_token("this.is.not.a.valid.jwt.token")
print(f"Invalid token result: {invalid_result}")
print("Returns None for invalid/expired tokens — safe to use in conditionals.")

## 4. Configurable Access Name

By default, the ORM uses `{table_name}_auth` as the access name. You can
customize this via `access_name` in `SurrealConfigDict`.

This is useful when:
- You have multiple access methods for the same table (e.g., `password_auth`, `oauth_auth`)
- Your access names follow a different convention
- You are integrating with an existing SurrealDB schema

```python
class User(AuthenticatedUserMixin, BaseSurrealModel):
    model_config = SurrealConfigDict(
        table_type=TableType.USER,
        access_name="account",  # Uses DEFINE ACCESS account ON DATABASE
    )
```

The `access_name` must match the name used in your `DEFINE ACCESS` statement.

In [ ]:
# Show how the access name is derived from model config
print("Access name configuration examples:")
print()

# Our User model uses a custom access_name
print(f"User model access_name: '{User.model_config.get('access_name', 'users_auth')}'")
print("  -> Maps to: DEFINE ACCESS account ON DATABASE")
print()

# If access_name is not set, the default would be:
table_name = User.model_config.get("table_name", "users")
print(f"Default access_name (if not configured): '{table_name}_auth'")
print(f"  -> Would map to: DEFINE ACCESS {table_name}_auth ON DATABASE")

## Cleanup

Remove all data and access definitions created during this notebook.

In [ ]:
# Clean up: remove users and the access definition
await User.raw_query("DELETE FROM users")
await User.raw_query("REMOVE ACCESS account ON DATABASE")

print("Cleanup complete: all users deleted and ACCESS definition removed.")

## Summary

| Method | Returns | Use Case |
|--------|---------|----------|
| `User.signup(...)` | `(user, token)` | User registration |
| `User.signin(...)` | `(user, token)` | User login |
| `User.authenticate_token(token)` | `(user, record_id)` or `None` | Full validation (fetches user) |
| `User.validate_token(token)` | `record_id` or `None` | Lightweight validation (no DB fetch) |

**Key points:**
- Authentication uses **ephemeral connections** — the root connection is never modified
- `Encrypted` fields are hashed by SurrealDB using argon2
- You must create a `DEFINE ACCESS` rule before using signup/signin
- The `access_name` in your model config must match the `DEFINE ACCESS` name